In [1]:
import csv
import openpyxl
import re
import types
import time
import datetime
import sys
#from types import NoneType
from datetime import date
from datetime import timezone
from datetime import timedelta
from leapYear import leapYear
from getFloodType import getFloodType
from getLatLon import getLatLon
from getWFOs import getWFOs

In [2]:
##########################
# Read in a file
##########################
while True:
    try:
        file_load = int(input("MPD number?: "))
        break
    except ValueError as e:
        print("Not a valid MPD number")
        
print(file_load)        

#
# Construct a file name from user input
#

path_to_file = '/Users/jameshayes/'
today = date.today()
yy = today.strftime("%Y")

file_name = f'{path_to_file}{yy}_mpd_{file_load}.txt'
print(file_name)

MPD number?: 1100
1100
/Users/jameshayes/2022_mpd_1100.txt


In [3]:
#read in the MPD
with open (file_name) as f:
    bx2 = f.readlines()
    str1 =''.join(bx2)
    #print(str1)

Mesoscale Precipitation Discussion 1100
NWS Weather Prediction Center College Park MD
810 PM EDT Sun Oct 16 2022

Areas affected...Southwest TX

Concerning...Heavy rainfall...Flash flooding possible

Valid 170009Z - 170609Z

Summary...Intense supercell and multicell clusters persist
across portions of South Central-Texas this evening along a cold
front, with upstream thunderstorm development noted over Northeast
Mexico. Flash flood chances are expected to increase tonight over
Southwest Texas as the activity over Mexico translates eastward
while the storms along the cold front propagate south.

Discussion...Latest GOES-16 IR imagery depicts an intense
rightward deviating supercell propagating eastward over the TX
Hill Country with downstream multi-cell clusters along a cold
front. Meanwhile, convection to the south in Northeast Mexico is
exhibhiting rapid cooling cloud tops as it moves eastward off the
terrain. The uptick in coverage is in response to increasing upper
divergence eviden

In [4]:
#############################################
# Get the valid times of the MPD
#############################################
ValidTime = re.search('Valid.*', str1)
if ValidTime:
    print(ValidTime.group(0))
    ValidValid = ValidTime.group(0)
        
ValidSplice = ValidValid[6:]
ValidStart = ValidSplice[2:6]
ValidEnd = ValidSplice[12:16]
ValidDay = ValidSplice[0:2]
ValidDay = int(ValidDay)

Valid 170009Z - 170609Z


In [5]:
#########################################
# Set up the discussion number and date
#########################################
import pytz
import datetime

#
# Parsing needed time and dat elements
#

meso1 = bx2[0]
meso2 = meso1[-5:]
date1 = bx2[2]
timeSpace = date1[-30:]
timeSpace = timeSpace.strip()
year = date1[-5:]
year1 = str(year[:4])
month = date1[15:19]
month = month.strip()
day1 = date1[19:22]
day1 = int((day1.strip()))
tod = date1[4:7]
tod = tod.strip()

#
# Set up a fix for times that are not four digits
#

timeFix = int(date1[:4])
if timeFix < 1000:
    timeFix = str(timeFix)
    timeFix = ("0" + timeFix)
    hh = int(timeFix[:2])
    mm = int(timeFix[2:4])
                
if tod == 'PM' and hh != 12:
    hh = hh + 12
elif tod == 'AM' and hh == 12:
    hh = 0

    
#
# Convert issuance time to UTC
#

dt_str = (f'{year1},{month},{day1},{hh},{mm}')
dt_obj = datetime.datetime.strptime(dt_str,'%Y,%b,%d,%H,%M')
dt_utc = str(dt_obj.astimezone(pytz.UTC))
dt_utc_month = (dt_str[5:8])

year_UTC = int(dt_utc[:4])
month_UTC = (dt_utc[5:7])
month_number_UTC = int(dt_utc[5:7])
day_UTC = int(dt_utc[8:11])
hour_UTC = int(dt_utc[11:13])
minute_UTC = int(dt_utc[14:16])
monthDay = (f'{dt_utc_month} {day_UTC}').strip()

#
# Set up datestring to convert to timestamp
#

startHour = ValidStart[0:2]
startMinute = ValidStart[2:4]
stopHour = ValidEnd[0:2]
stopMinute = ValidEnd[2:4]

In [6]:
floodtype = getFloodType(str1)

possible
The flood type is possible


In [7]:
LatLon1 = getLatLon(str1)

LAT...LON   31990083 31869913 30549868 27829901 27520015
            29390114 29250311 29340390 29690457 30340490
            31200457 31700320


In [8]:
FinalWFO = getWFOs(str1)

The final WFO list is  CRP EWX MAF SJT


In [9]:
##############################################
# Extract and process the Lat/Lon information
##############################################

LatLon2 = LatLon1[10:]
interim = LatLon2.strip('\n')
interim2 = re.split(r'\s{1,}', interim)

latlat = [None] * len(interim2)
lonlon = [None] * len(interim2)

latlatX = [None] * len(interim2)
lonlonX = [None] * len(interim2)
lonlonFinal = [None] * len(interim2)

lat = [len(interim2)]
lon = [len(interim2)]  

i =  1
while i < len(interim2):
    latlat[i] = interim2[i][:4]
    lonlon[i] = interim2[i][4:]
    latlatX[i] = (float(latlat[i])/100)
    lonlonX[i] = (float(lonlon[i])/100)
    if lonlonX[i] < 30:
       lonlonFinal[i] = ((lonlonX[i] + 100)*-1)
    else:
       lonlonFinal[i] = ((lonlonX[i])*-1)
    
       print(i, latlatX[i], lonlonFinal[i])       
    i = i + 1

2 31.86 -99.13
3 30.54 -98.68
4 27.82 -99.01


In [10]:
#
# Is this a leap year?
#

leap_year = leapYear(year_UTC)
print(leap_year)

#
# Determining the number of days in the month
# 

if month_number_UTC in (1, 3, 5, 7, 8, 10, 12):
    month_length = 31
elif month_number_UTC == 2:
    if leap_year:
        month_length = 29
    else:
        month_length = 28
else:
    month_length = 30
   
#
# Calculating beginning and end of the MPD
#

stopHour = int(stopHour) 
startHour = int(startHour)
print(f'The stop hour is {startHour} and the start hour is {stopHour}')

if stopHour < startHour:
    day_UTC += 1    

if day_UTC == month_length and startHour < 3:  
    day_UTC = 1 
            
if day_UTC == 1:
    if month_number == 12:
        month_number = 1
        year_UTC += 1
    else:
        month_number += 1
  

#
# Create the date string to compute the datestamp
#

dateStringStart = (f'{month_number_UTC}/{day_UTC}/{year_UTC} {startHour}:{startMinute}');print(dateStringStart)
dateStringEnd = (f'{month_number_UTC}/{day_UTC}/{year_UTC} {stopHour}:{stopMinute}');print(dateStringEnd)

False
The stop hour is 0 and the start hour is 6
10/17/2022 0:09
10/17/2022 6:09


In [11]:
#
# setup the time stamps
#

import time
import datetime
from datetime import datetime

print(dateStringStart)
objStart = time.strptime(dateStringStart, "%m/%d/%Y %H:%M")
objStop = time.strptime(dateStringEnd,"%m/%d/%Y %H:%M")


tStampStart = time.mktime(objStart);print(tStampStart)
tStampStop = time.mktime(objStop);print(tStampStop)

diffTime = ((tStampStop - tStampStart)/60);print(diffTime)
diffTime = int(diffTime)

10/17/2022 0:09
1665979740.0
1666001340.0
360.0


In [12]:
######x###########################################
# Write the other information to an Excel file
##################################################
print(FinalWFO)

from openpyxl import load_workbook
qb22 = int(file_load)
print(qb22)
qb33 = qb22 + 1
print(qb33)


file_path_xls = f'{path_to_file}MPD{yy}.xlsx'
print(file_path_xls)


wb = load_workbook(file_path_xls)
print(wb)
sheet = wb.active

# write out the header

print("Writing out the header to the Excel file...")

a1 = sheet['A1']
a1.value = "MPD#"
b1 = sheet['B1']
b1.value = "Date"
c1 = sheet['C1']
c1.value = "Cat"
d1 = sheet['D1']
d1.value = "Start"
e1 = sheet['E1']
e1.value = "Stop"
f1 = sheet['F1']
f1.value = "Time"
g1 = sheet['G1']
g1.value = "Area"
h1 = sheet['H1']
h1.value = "WFO"

# Write the data to the correct locations

print
print("Writing the MPD info to the file...")

# First the MPD#
aNext = sheet.cell(row = qb33, column = 1)
aNext.value = qb22

#Next the date
bNext = sheet.cell(row = qb33, column = 2)
bNext.value = monthDay

#Next the category of flooding
cNext = sheet.cell(row = qb33, column = 3)
cNext.value = floodtype

# Start time of the MPD
dNext = sheet.cell(row = qb33, column = 4)
dNext.value = ValidStart

# End of the MPD
eNext = sheet.cell(row = qb33, column = 5)
eNext.value = ValidEnd

# Get the time length of the MPD (under construction)
fNext = sheet.cell(row = qb33, column = 6)
fNext.value = diffTime

# Get the Area (under construction)
#gNext sheet.cell(row = qb33, column = 7)
#gNext.value = Area

# WFOs in the MPD
hNext = sheet.cell(row = qb33, column = 8)
hNext.value = FinalWFO


wb.save(file_path_xls)

CRP EWX MAF SJT
1100
1101
/Users/jameshayes/MPD2022.xlsx
Writing out the header to the Excel file...
Writing the MPD info to the file...
